## Training Guarani ASR

In [1]:
from utils_guarani import *
from huggingface_hub import notebook_login
import json
from transformers import Wav2Vec2CTCTokenizer

notebook_login()

In [2]:
from datasets import load_dataset, load_metric, Audio

voice_train = load_dataset("mozilla-foundation/common_voice_8_0", "gn", use_auth_token=True,split="train+validation+test")

voice_train=voice_train.train_test_split(train_size=0.9)
voice_test=voice_train["test"]
voice_train=voice_train["train"]

Reusing dataset common_voice (/home/jhonparra/.cache/huggingface/datasets/mozilla-foundation___common_voice/gn/8.0.0/b8bc4d453193c06a43269b46cd87f075c70f152ac963b7f28f7a2760c45ec3e8)
Loading cached split indices for dataset at /home/jhonparra/.cache/huggingface/datasets/mozilla-foundation___common_voice/gn/8.0.0/b8bc4d453193c06a43269b46cd87f075c70f152ac963b7f28f7a2760c45ec3e8/cache-9a03735d214bf2a9.arrow and /home/jhonparra/.cache/huggingface/datasets/mozilla-foundation___common_voice/gn/8.0.0/b8bc4d453193c06a43269b46cd87f075c70f152ac963b7f28f7a2760c45ec3e8/cache-5d9db649fb70176c.arrow


### Listen to audio

In [3]:
import IPython.display as ipd
import numpy as np
import random


samp=random.randint(0,voice_train.shape[0])
print(samp)
ipd.Audio(data=voice_train[samp]["audio"]["array"], autoplay=True, rate=voice_train[samp]["audio"]["sampling_rate"])

275


### Normalize Data

In [5]:
from IPython.display import display, HTML
show_random_elements_Dataset(voice_train,5)

,client_id,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,392c88536e58bbcc83d886628443d91d8c066c66194a47c721f3ed98624c786c2d63793b48f211ce3c028d62e250b1b64b05a703ec5cef20216a0e718f42fa7a,"""upépe omohu'ãkuri mbo'esyry mbohapyha""",2,0,twenties,male,,gn,
1,392c88536e58bbcc83d886628443d91d8c066c66194a47c721f3ed98624c786c2d63793b48f211ce3c028d62e250b1b64b05a703ec5cef20216a0e718f42fa7a,"""Sapy'apy'a ojejuhu upépe Pláta Yvyguy.""",2,0,,,,gn,
2,392c88536e58bbcc83d886628443d91d8c066c66194a47c721f3ed98624c786c2d63793b48f211ce3c028d62e250b1b64b05a703ec5cef20216a0e718f42fa7a,"""Oiko ka'aguýre.""",2,0,,,,gn,
3,9637a6db1fd6e7ec19b6718cf85d53db603dead13eb6bb35743afca2fcb7427146d6a2cb0e367f36a0432985a588e01deb76cdfbf006b034d08abc835c079a85,"""Oñomongeta kyre'ỹ ha ipahaitépe katu ombojoaju hikuái hekove.""",2,0,,,,gn,
4,9637a6db1fd6e7ec19b6718cf85d53db603dead13eb6bb35743afca2fcb7427146d6a2cb0e367f36a0432985a588e01deb76cdfbf006b034d08abc835c079a85,"""Pajagua mascada ha pastel mandi'o""",2,0,,,,gn,


In [5]:
##generate preprocessing
import re

def normalizing_text(batch):
    import unicodedata
    method = 'NFKC' 
    batch["sentence"] = "".join(str(unicodedata.normalize(method,i)) for i in batch["sentence"])
    return batch
    
chars_to_remove = '[\"\“\”\�\!\#\$\%\&\(\)\*\+\,\-\.\/\:\;\<\=\>\u003F\@\^\_\{\|\}]'
def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove, '', batch["sentence"]).lower()
    batch["sentence"] = re.sub("”", '', batch["sentence"])
    batch["sentence"] = re.sub("“", '', batch["sentence"])
    batch["sentence"] = re.sub("’","ʼ",batch["sentence"])
    return batch

def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [20]:
train_data_pr=voice_train.map(remove_special_characters)
test_data_pr=voice_test.map(remove_special_characters)
train_data_pr=train_data_pr.map(normalizing_text)
test_data_pr=test_data_pr.map(normalizing_text)
print("AFTER TEXT PROCESSING")
show_random_elements_Dataset(train_data_pr,10)


Loading cached processed dataset at /home/jhonparra/.cache/huggingface/datasets/mozilla-foundation___common_voice/gn/8.0.0/b8bc4d453193c06a43269b46cd87f075c70f152ac963b7f28f7a2760c45ec3e8/cache-67b1232ba7f31fed.arrow
Loading cached processed dataset at /home/jhonparra/.cache/huggingface/datasets/mozilla-foundation___common_voice/gn/8.0.0/b8bc4d453193c06a43269b46cd87f075c70f152ac963b7f28f7a2760c45ec3e8/cache-b078ae2196e129b6.arrow


0ex [00:00, ?ex/s]

Loading cached processed dataset at /home/jhonparra/.cache/huggingface/datasets/mozilla-foundation___common_voice/gn/8.0.0/b8bc4d453193c06a43269b46cd87f075c70f152ac963b7f28f7a2760c45ec3e8/cache-1f81cdd5909a9f5b.arrow


AFTER TEXT PROCESSING


,client_id,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,cdcf94339b63b1d5fc33da362e57a279149224ff2a5ab51bcab1b34021d9853179785659ffd9c5cf86d2f53b6037f29658023f41f0fa507eb4125b481d98b96a,tekokatu térã tekomarã heñói ndénte reipotágui,2,1,thirties,female,,gn,
1,392c88536e58bbcc83d886628443d91d8c066c66194a47c721f3ed98624c786c2d63793b48f211ce3c028d62e250b1b64b05a703ec5cef20216a0e718f42fa7a,ko léi ojepurúta ñemoarandu ha ñemosãsorã,2,0,twenties,male,,gn,
2,20aef279a05c48023dd23e6df1d5c34f65b3f9db5dcd5c5926109ab7d2cbb2624dc9390976952321723a5a965f1c92e8b37ae49f2eaf553f01bae2b4978607ad,jahechángo ogaygua ápe ha pépe oku'e,2,0,twenties,male,,gn,
3,392c88536e58bbcc83d886628443d91d8c066c66194a47c721f3ed98624c786c2d63793b48f211ce3c028d62e250b1b64b05a703ec5cef20216a0e718f42fa7a,mbaʼéichapa héra nde sy,2,0,twenties,male,,gn,
4,20aef279a05c48023dd23e6df1d5c34f65b3f9db5dcd5c5926109ab7d2cbb2624dc9390976952321723a5a965f1c92e8b37ae49f2eaf553f01bae2b4978607ad,mokõi kotymi ha peteĩ tataypy,2,0,twenties,male,,gn,
5,4e5e68b12f3cf3fd71a400acc196f53f41eb3d81880e857a2e46a98dd29fc3008e638136b8df9ee6c19216cab91087c81b0fe395f799e3a8e73e3e24e4abc0a3,avei oipuru yvyrapã ha hu'y,2,0,,,,gn,
6,cdcf94339b63b1d5fc33da362e57a279149224ff2a5ab51bcab1b34021d9853179785659ffd9c5cf86d2f53b6037f29658023f41f0fa507eb4125b481d98b96a,mokõive ndogueroviaséi upe mba'e ojehuva'ekue,2,0,thirties,female,,gn,
7,29e005ebccc97df3fba4c3479544c44321c3cc9cd1632dbe2e874da925ce36c83a46e713d43aa7d0aef74ce800382f4c884efe088614b2d4efbb609c6d5218dd,ijeike políticape,2,0,,,,gn,
8,2b2b3bf1b7004f4cdb7f47f1c88f55f126dba5a934d41c8afc027f15415d9a2c17d2c83ed9c751f5d7d22ed2341b2940869aa5eb7b6b47539e8c00d0c6faa411,karai ferreira ombosako'i irambosarã,2,0,,,,gn,
9,563d3d204eb15d05a35315360f159466ebeb253a27497d847f46aa396ab826fe95e5b8482440cf613d7581612f53930543babdfeb574fbfb2638719c708fbe43,ovákuri ipehẽnguekuéra ndive táva paraguaýpe,2,0,,,,gn,


In [21]:
vocab_train = train_data_pr.map(extract_all_chars, batched=True, batch_size=-1, remove_columns=train_data_pr.column_names)
vocab_test = test_data_pr.map(extract_all_chars, batched=True, batch_size=-1, remove_columns=test_data_pr.column_names)

vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
print(vocab_dict)

  0%|          | 0/1 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/jhonparra/.cache/huggingface/datasets/mozilla-foundation___common_voice/gn/8.0.0/b8bc4d453193c06a43269b46cd87f075c70f152ac963b7f28f7a2760c45ec3e8/cache-acb4163aaefc6fa8.arrow


{' ': 0, "'": 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'y': 24, 'z': 25, '¡': 26, 'á': 27, 'ã': 28, 'é': 29, 'í': 30, 'ñ': 31, 'ó': 32, 'õ': 33, 'ú': 34, 'ý': 35, 'ĩ': 36, 'ũ': 37, 'ʼ': 38, '̃': 39, 'ẽ': 40, 'ỹ': 41}


In [8]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
##add unknown and padding tokens
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

44

In [11]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)


tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

file ./config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
repo_name = "glob-asr/wav2vec2-large-xls-r-300m-guarani-small"
tokenizer.push_to_hub(repo_name)

/home/jhonparra/.local/lib/python3.8/site-packages/huggingface_hub/hf_api.py:715: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  warnings.warn(
/home/jhonparra/.local/lib/python3.8/site-packages/huggingface_hub/hf_api.py:715: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  warnings.warn(
Cloning https://huggingface.co/jhonparra18/wav2vec2-large-xls-r-300m-guarani-small into local empty directory.
To https://huggingface.co/jhonparra18/wav2vec2-large-xls-r-300m-guarani-small
   2f0070c..04c520a  main -> main



'https://huggingface.co/jhonparra18/wav2vec2-large-xls-r-300m-guarani-small/commit/04c520af7cfcfb69f9a8b4e0908483584ce597d0'

### Model Definition & Audio Processing

In [22]:
from transformers import Wav2Vec2FeatureExtractor,Wav2Vec2Processor
 
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [23]:
train_data_pr = train_data_pr.cast_column("audio", Audio(sampling_rate=16000))
test_data_pr = test_data_pr.cast_column("audio", Audio(sampling_rate=16000))

In [25]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=16000).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [26]:
train_data_pr = train_data_pr.map(prepare_dataset, remove_columns=train_data_pr.column_names)
test_data_pr = test_data_pr.map(prepare_dataset, remove_columns=test_data_pr.column_names)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [27]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

wer_metric = load_metric("wer")
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Downloading:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

In [29]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
     hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

model.freeze_feature_extractor()

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.bias', 'project_q.weight', 'project_hid.bias', 'project_q.bias', 'project_hid.weight', 'quantizer.codevectors', 'quantizer.weight_proj.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it 

### Set up trainer

In [30]:
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=10,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=500,
  eval_steps=200,
  logging_steps=200,
  learning_rate=2e-4,
  warmup_steps=100,
  save_total_limit=20,
  push_to_hub=True,
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data_pr,
    eval_dataset=test_data_pr,
    tokenizer=processor.feature_extractor,
)

/home/jhonparra/.local/lib/python3.8/site-packages/huggingface_hub/hf_api.py:715: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  warnings.warn(
/home/jhonparra/hf-asr-comp/guarani_language/glob-asr/wav2vec2-large-xls-r-300m-guarani-small is already a clone of https://huggingface.co/jhonparra18/wav2vec2-large-xls-r-300m-guarani-small. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp fp16 backend


In [31]:
trainer.train()


The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running training *****
  Num examples = 490
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 310
/home/jhonparra/.local/lib/python3.8/site-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/home/jhonparra/.local/lib/python3.8/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently ro

Step,Training Loss,Validation Loss


RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.